# Threads and Queues

In [1]:
import threading
import time

def countdown():
    for second in range (5, 0, -1):
        print(f"{second}!", end = " ")
        time.sleep(1)
    print("Happy New Year!")

countdown_thread = threading.Thread(target=countdown)
countdown_thread.start()
countdown_thread.join()

5! 4! 3! 2! 1! Happy New Year!


### Passing Arguments to a Thread Target

In [1]:
import threading
import time

def countdown(secs):
    for second in range(secs, 0, -1):
        print(f"{second}!", end = " ")
        time.sleep(1)
    print("Happy New Year!")

countdown_thread = threading.Thread(target=countdown, args=(3,))
countdown_thread.start()
countdown_thread.join()

3! 2! 1! Happy New Year!


In [2]:
import threading
import time

def countdown(secs):
    for second in range(secs, 0, -1):
        print(f"{second}!", end = " ")
        time.sleep(1)
    print("Happy New Year!")

countdown_thread = threading.Thread(target=countdown, kwargs=dict(secs=3))
countdown_thread.start()
countdown_thread.join()

3! 2! 1! Happy New Year!


## Multiple Threads

In [3]:
import threading
import time

def countdown(secs, punc="!"):
    for second in range(secs, 0, -1):
        print(f"{second}{punc}", end = " ")
        time.sleep(1)
    print(f"Happy New Year{punc}", end=" ")

t1 = threading.Thread(target=countdown, args=[3], name="First")
t2 = threading.Thread(target=countdown, kwargs=({"secs":3,"punc":"?"}), name="Second")
t1.start()
time.sleep(.5)
t2.start()
t1.join()
t2.join()

3! 3? 2! 2? 1! 1? Happy New Year! Happy New Year? 

## Passing Data Between Threads

In [5]:
from pyuvm import Singleton
import threading

class Conduit(metaclass=Singleton):
    def __init__(self):
        self.numb = None
        
    def put_numb(self,nn):
        self.numb = nn
        
    def get_numb(self):
        return self.numb

def producer(max_numb):
    for nn in range(max_numb, -1, -1):
        print(f"sending {nn}", end = " ")
        Conduit().put_numb(nn)

def consumer():
    numb = 10
    while numb is None or numb > 0:
        numb = Conduit().get_numb()
        if numb is not None:
            print(f"got {numb}", end = " ")

pthread = threading.Thread(target=producer, kwargs=dict(max_numb=3))
cthread = threading.Thread(target=consumer)
pthread.start()
cthread.start()
pthread.join()
cthread.join()

sending 3 sending 2 sending 1 sending 0 got 0 

In [5]:
from pyuvm import Singleton
import threading
from queue import Queue

class Conduit(metaclass=Singleton):
    def __init__(self):
        self.qq = Queue(maxsize=1)

def producer(max_numb):
    for nn in range(max_numb, -1, -1):
        Conduit().qq.put(nn)
        print(f"sent {nn}", end = " ")


def consumer():
    numb = 10
    while numb > 0:
        numb = Conduit().qq.get()
        print(f"got {numb}", end = " ")

pthread = threading.Thread(target=producer,  kwargs=dict(max_numb=3))
cthread = threading.Thread(target=consumer)
pthread.start()
cthread.start()
pthread.join()
cthread.join()

sent 3got 3 sent 2  got 2 sent 1 got 1 sent 0 got 0 

## Using Thread Conditions to Block

The race car example

In [6]:
from pyuvm import Singleton
import threading

class FlagHolder(metaclass=Singleton):
    def __init__(self, cond=None):
        self.flag = False
        self.cond = cond
        self.set_count = 1

    def set_flag(self, flag):
        print(f"\nSetting flag to {flag}")
        self.set_count += 1
        self.flag = flag
        with self.cond:
            self.cond.notify_all()

def check_flag():
    print(f"Checking {FlagHolder().set_count}", end=" ")
    return FlagHolder().flag

def racecar(number, flag_cond):
    with flag_cond:
        flag_cond.wait_for(check_flag)
    print(f"# {number:02d} Go!", end=" ")

flag_cond = threading.Condition()
flag_holder = FlagHolder(flag_cond)
thread_list=[]

for ii in range(10,13):
    thread = threading.Thread(target=racecar, args=[ii, flag_cond])
    thread_list.append(thread)
for tt in thread_list:
    tt.start()


flag_holder.set_flag(False)
time.sleep(0.1)
flag_holder.set_flag(False)
time.sleep(0.1)
flag_holder.set_flag(True)

Checking 1 Checking 1 Checking 1
Setting flag to False
 Checking 2 Checking 2 Checking 2 
Setting flag to False
Checking 3 Checking 3 Checking 3 
Setting flag to True
Checking 4 # 12 Go! Checking 4 # 10 Go! Checking 4 # 11 Go! 

## Ending Threads
### Terminating Data

In [7]:
import threading

def producer(numb, qq):
    for nn in range(numb):
        qq.put(nn)
        print(f"sent {nn}", end=" ")
    qq.put(None)

def consumer(qq):
    nn = qq.get()
    while nn is not None:
        print(f"got {nn}", end=" ")
        nn = qq.get()

queue = Queue(maxsize=1)
pthread = threading.Thread(target=producer, args=[4, queue])
cthread = threading.Thread(target=consumer, args=[queue])
pthread.start()
cthread.start()


sent 0 got 0sent 1  got 1 sent 2 got 2 sent 3 got 3 

### Termination Flag

In [8]:
import threading
from pyuvm import Singleton
import time
from queue import Queue, Empty

class Finished(metaclass=Singleton):
    def __init__(self):
        self.flag = False

def producer(numb, qq):
    for nn in range(numb):
        qq.put(nn)
        print(f"sent {nn}", end=" ")
        time.sleep(0.2)
    Finished().flag = True

def consumer(qq):
    while not Finished().flag:
        try:
            nn = qq.get(timeout=0.1)
            print(f"got {nn}", end=" ")
        except Empty:
            pass


queue = Queue(maxsize=1)
pthread = threading.Thread(target=producer, args=[4, queue])
cthread = threading.Thread(target=consumer, args=[queue])
pthread.start()
cthread.start()

sent 0 got 0 sent 1got 1  sent 2 got 2 sent 3got 3  